# INITIALIZATION

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## UNCOMMENT TO CLONE APLHAPOSE SOURCE CODE

In [ ]:
# !git clone -b pytorch https://github.com/MVIG-SJTU/AlphaPose.git

In [ ]:
%cd /content/gdrive/My Drive/Alphapose

/content/gdrive/My Drive/Alphapose


In [ ]:
%cd AlphaPose

/content/gdrive/My Drive/Alphapose/AlphaPose


## PIP INSTALL

In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install visdom

     |████████████████████████████████| 686kB 17.9MB/s 
     |████████████████████████████████| 204kB 52.4MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-cp36-none-any.whl size=655251 sha256=80c8e43b1db5a74653e41382d1c5aa80f5efb425fa1a31c50ed07028cf592373
  Stored in directory: /root/.cache/pip/wheels/70/19/a7/6d589ed967f4dfefd33bc166d081257bd4ed0cb618dccfd62a
  Created wheel for torchfile: filename=torchfile-0.1.0-cp36-none-any.whl size=5711 sha256=bb825041f0c48a4c561ef3f48faaf89c78afc8d8859b59d7a1f7d8469dfab699
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built visdom torchfile


# TO IDENTIFY HUMANS USING ALPHA POSE AND EXTRACT KEYPOINTS

## GENERATE ALPHA POSE RESULTS AS JSON ON VIDEO

CONFIDENCE AND NMS CAN BE VARIED 

In [ ]:
!python3 video_demo.py --video $"/content/gdrive/MyDrive/Alphapose/AlphaPose/examples/Video/1047598234-preview.mp4" --outdir examples/res/random_videos/1047598234 --conf 0.8 --nms 0.45 
# --inp_dim 480 --detbatch 2

Loading YOLO model..
Loading pose model from ./models/sppe/duc_se.pth
100% 456/456 [00:44<00:00, 10.23it/s]
===========================> Finish Model Running.
[W CudaIPCTypes.cpp:22] Producer process has been terminated before all shared CUDA tensors released. See Note [Sharing CUDA tensors]


## GENERATE ALPHA POSE RESULTS AS JSON AS WELL AS SAVE RENDERED VIDEO TO ANALYZE

In [ ]:
!python3 video_demo.py --video $"/content/gdrive/MyDrive/Alphapose/AlphaPose/examples/Video/trimmed_walk.mp4" --outdir /content/gdrive/MyDrive/Alphapose/AlphaPose/examples/res/random_videos/1046171221/ --save_video  --matching  --conf 0.7 --nms 0.5

Loading YOLO model..
===========================> This video get 211 frames in total.
Loading pose model from ./models/sppe/duc_se.pth
100% 211/212 [00:17<00:00, 10.95it/s]===========================> Finish Model Running.
===========================> Rendering remaining images in the queue...
===========================> If this step takes too long, you can enable the --vis_fast flag to use fast rendering (real-time).
100% 211/212 [00:37<00:00,  5.69it/s]
[W CudaIPCTypes.cpp:22] Producer process has been terminated before all shared CUDA tensors released. See Note [Sharing CUDA tensors]


## GENERATE ALPHA POSE RESULTS AS JSON FOR IMAGES

In [ ]:
!python3 demo.py --indir $"/content/gdrive/My Drive/Alphapose/AlphaPose/examples" --outdir examples/res 

## GENERATE ALPHA POSE RESULTS AS JSON FOR IMAGES AND SAVE THE RESULTS FOR ANALYSIS

In [ ]:
!python3 demo.py --list examples/list-coco-demo.txt --indir $"/content/gdrive/My Drive/Alphapose/AlphaPose/examples/demo" --outdir examples/res --save_img


# EXTRACT FACE FROM POSE

Automated Code with inclusion of Model to detect Mask

In [ ]:
# !cd /content/gdrive/MyDrive/Alphapose/AlphaPose/examples/res

In [ ]:
# !find . -name "frame*" 
# -delete

## FUNCTION TO CALL

In [ ]:
# %%writefile detect_mask_image.py
# USAGE
# python detect_mask_image.py --image images/pic1.jpeg

# import the necessary packages
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import argparse
import cv2
from google.colab.patches import cv2_imshow
import os

def mask_image_faster(image_input):

	model = load_model("/content/gdrive/MyDrive/Alphapose/AlphaPose/MaskDetectionModel/mask_detector_with_blur.model")
  
	image = image_input
	
	print("[INFO] computing face detections...")

	face = image
	face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
	face = cv2.resize(face, (224, 224))
	face = img_to_array(face)
	face = preprocess_input(face)
	face = np.expand_dims(face, axis=0)

	(mask, withoutMask) = model.predict(face)[0]

	label = "Mask" if mask > withoutMask else "No Mask"
	
	label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100);

	return label


## EXTRACT FACE WITH HEURISTIC 1 - THIS TRIAL WAS IGNORED AS IT DIDNT PRODUCE BEST RESULTS.

In [ ]:
###Automated code
import cv2
import json
import math

with open('/content/gdrive/MyDrive/Alphapose/AlphaPose/examples/res/random_videos/1048231489/alphapose-results.json') as f:
  data = json.load(f)

print(len(data))
from PIL import Image, ImageDraw
total_persons_from_all_frames=len(data)
from google.colab.patches import cv2_imshow
vidcap = cv2.VideoCapture('/content/gdrive/MyDrive/Alphapose/AlphaPose/examples/Video/1048231489-preview.mp4')
success,image = vidcap.read()
count = 0
iter=0
json_count=0
exception_count=0
window_name = 'Image'
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
thickness = 2
color = (255, 0, 0) 
# while success:
while success :
  cv2.imwrite("frame%d.jpg" % count, image)     
 

  prev_person=data[json_count]
  person=data[json_count]

  print(person['image_id'],prev_person['image_id'],"frame%d.jpg"%count)
  while person['image_id']==prev_person['image_id']:
        leye_x=person['keypoints'][1*3]
        leye_y=person['keypoints'][(1*3)+1]
        # rshoulder_x=person['keypoints'][6*3]-20
        rshoulder_x=person['keypoints'][6*3]
        rshoulder_y=person['keypoints'][(6*3)+1]
        lshoulder_x=person['keypoints'][5*3]
        lshoulder_y=person['keypoints'][(5*3)+1]
        x1=math.floor(leye_x+20)
        y1=math.floor(leye_y-(rshoulder_y-leye_y+20))
        x2=math.floor(rshoulder_x-20)
        y2=math.floor(rshoulder_y)
     
        # start_point = (x1,y1) 
        # end_point = (x2,y2)
        if y1<y2:
            start_point = (x1,y1) 
      
            end_point = (x2,y2)
        elif y1>y2:
            start_point = (x1,y2) 
     
            end_point = (x2,y1)

        try:
          crop_img = image[ y1:y2,x2:x1]
          
          label=(mask_image_faster(crop_img))
          if 'No Mask' in label:
              color=(0, 0,255) 
          else:
              color=(0, 255,0) 
          image = cv2.rectangle(image, start_point, end_point, color, thickness)
          org = (x1, y1) 
    
          image = cv2.putText(image, label, org, font,  
                    fontScale, color, thickness, cv2.LINE_AA)
        except Exception as err:
          print(err, str(start_point), str(end_point),"x1-->y1",str(x1),str(y1),"x2-->y2",str(x2),str(y2))
          cv2_imshow( image)
          exception_count+=1
       
        print('json count'+ str(json_count))
        json_count+=1

        
        prev_person=person
        try:
          person=data[json_count]
        except:
          break
  
 
  iter+=1
  count += 1
  print('exception count is' + str(exception_count))

  cv2.imwrite("/content/gdrive/MyDrive/Alphapose/AlphaPose/examples/res/Output_Frames/frame%d.jpg" % count, image)
  success,image = vidcap.read()
  
print(count)


## EXTRACT FACE WITH HEURISTIC 2 - THIS TRIAL WAS IGNORED AS IT DIDNT PRODUCE BEST RESULTS.

In [ ]:
###Automated code
import cv2
import json
import math

with open('/content/gdrive/MyDrive/Alphapose/AlphaPose/examples/res/random_videos/1047598234/alphapose-results.json') as f:
  data = json.load(f)

print(len(data))
from PIL import Image, ImageDraw

from google.colab.patches import cv2_imshow
vidcap = cv2.VideoCapture('/content/gdrive/MyDrive/Alphapose/AlphaPose/examples/Video/1047598234-preview.mp4')
success,image = vidcap.read()
count = 0
iter=0
json_count=0

window_name = 'Image'
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
thickness = 2
color = (255, 0, 0) 
# while success:
while success :
  cv2.imwrite("frame%d.jpg" % count, image)     
 

  prev_person=data[json_count]
  person=data[json_count]

  print(person['image_id'],prev_person['image_id'],"frame%d.jpg"%count)
  while person['image_id']==prev_person['image_id']:
        leye_x=person['keypoints'][1*3]
        leye_y=person['keypoints'][(1*3)+1]
        # rshoulder_x=person['keypoints'][6*3]-20
        rshoulder_x=person['keypoints'][6*3]
        rshoulder_y=person['keypoints'][(6*3)+1]
        lshoulder_x=person['keypoints'][5*3]
        lshoulder_y=person['keypoints'][(5*3)+1]
        x1=math.floor(lshoulder_x)
        y1=math.floor(lshoulder_y)
        x2=math.floor(rshoulder_x)
        y2=math.floor(rshoulder_y)
        start_point = (x1,math.floor(y1+(x2-x1)+(0.2*(x2-x1))))
        end_point = (x2,y2)


        # start_point = (x1,y1) 
        # end_point = (x2,y2)
        # if y1<y2:
        #     start_point = (x1,y1) 
      
        #     end_point = (x2,y2)
        # elif y1>y2:
        #     start_point = (x1,y2) 
     
        #     end_point = (x2,y1)

        try:
          crop_img = image[ y1:y2,x2:x1]
          
          label=(mask_image_faster(crop_img))
          if 'No Mask' in label:
              color=(0, 0,255) 
          else:
              color=(0, 255,0) 
          image = cv2.rectangle(image, start_point, end_point, color, thickness)
          org = (x1, y1) 
    
          image = cv2.putText(image, label, org, font,  
                    fontScale, color, thickness, cv2.LINE_AA)
        except Exception as err:
          print(err)
        # if iter>6:
        try:
            # print(start_point,end_point,color,thickness)
            # cv2_imshow( crop_img)
            cv2.imwrite('/content/sample_data/cropped_images/crop_img_'+str(json_count)+'.jpg', crop_img) 
        except Exception as ex:
            print(ex)
            # continue
        print('json count'+ str(json_count))
        json_count+=1

        
        prev_person=person
        try:
          person=data[json_count]
        except:
          break
  
 
  iter+=1
  count += 1
  cv2_imshow(image)
  cv2.imwrite("/content/sample_data/Output_Frames/frame%d.jpg" % count, image)
  success,image = vidcap.read()
  
print(count)

## EXTRACT FACE WITH HEURISTIC 3 - THIS TRIAL WAS USED AS IT PRODUCES BEST RESULTS

In [ ]:
###Automated code
import cv2
import json
import math

with open('/content/gdrive/MyDrive/Alphapose/AlphaPose/examples/res/random_videos/1048231489/alphapose-results.json') as f:
  data = json.load(f)

print(len(data))
from PIL import Image, ImageDraw
total_persons_from_all_frames=len(data)
from google.colab.patches import cv2_imshow
vidcap = cv2.VideoCapture('/content/gdrive/MyDrive/Alphapose/AlphaPose/examples/Video/1048231489-preview.mp4')
success,image = vidcap.read()
count = 0
iter=0
json_count=0
exception_count=0
window_name = 'Image'
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
thickness = 2
color = (255, 0, 0) 
# while success:
while success :
  cv2.imwrite("frame%d.jpg" % count, image)     
 

  prev_person=data[json_count]
  person=data[json_count]

  print(person['image_id'],prev_person['image_id'],"frame%d.jpg"%count)
  while person['image_id']==prev_person['image_id']:
        leye_x=person['keypoints'][1*3]
        leye_y=person['keypoints'][(1*3)+1]
        # rshoulder_x=person['keypoints'][6*3]-20
        rshoulder_x=person['keypoints'][6*3]
        rshoulder_y=person['keypoints'][(6*3)+1]
        lshoulder_x=person['keypoints'][5*3]
        lshoulder_y=person['keypoints'][(5*3)+1]
        # x1=math.floor(leye_x+20)
        # x1=math.floor(lshoulder_x+20)
        x1=math.floor(lshoulder_x)
        # y1=math.floor(leye_y-(rshoulder_y-leye_y+20))
        y1=math.floor(leye_y-(rshoulder_y-leye_y))
        # x2=math.floor(rshoulder_x-20)
        x2=math.floor(rshoulder_x)
        y2=math.floor(rshoulder_y)
     
        # start_point = (x1,y1) 
        # end_point = (x2,y2)
        if y1<y2:
            start_point = (x1,y1) 
      
            end_point = (x2,y2)
        elif y1>y2:
            start_point = (x1,y2) 
     
            end_point = (x2,y1)

        try:
          crop_img = image[ y1:y2,x2:x1]
          
          label=(mask_image_faster(crop_img))
          if 'No Mask' in label:
              color=(0, 0,255) 
          else:
              color=(0, 255,0) 
          image = cv2.rectangle(image, start_point, end_point, color, thickness)
          org = (x1, y1) 
    
          image = cv2.putText(image, label, org, font,  
                    fontScale, color, thickness, cv2.LINE_AA)
        except Exception as err:
          print(err, str(start_point), str(end_point),"x1-->y1",str(x1),str(y1),"x2-->y2",str(x2),str(y2))
          cv2_imshow( image)
          exception_count+=1
        # if iter>6:
        # try:
        #     # print(start_point,end_point,color,thickness)
        #     # cv2_imshow( crop_img)
        #     cv2.imwrite('/content/sample_data/cropped_images/crop_img_'+str(json_count)+'.jpg', crop_img) 
        # except Exception as ex:
        #     print(ex)
        #     exception_count+=1
        #     # continue
        print('json count'+ str(json_count))
        json_count+=1

        
        prev_person=person
        try:
          person=data[json_count]
        except:
          break
  
 
  iter+=1
  count += 1
  print('exception count is' + str(exception_count))
#   cv2_imshow(image)
  cv2.imwrite("/content/gdrive/MyDrive/Alphapose/AlphaPose/examples/res/Output_Frames/frame%d.jpg" % count, image)
  success,image = vidcap.read()
  
print(count)


In [ ]:
#  !rm -r /content/gdrive/MyDrive/Alphapose/AlphaPose/examples/res/Output_Frames

# REGENERATE VIDEO FROM FRAMES.

In [ ]:
import re

def tryint(s):
    try:
        return int(s)
    except ValueError:
        return s
    
def alphanum_key(s):
    """ Turn a string into a list of string and number chunks.
        "z23a" -> ["z", 23, "a"]
    """
    return [ tryint(c) for c in re.split('([0-9]+)', s) ]

def sort_nicely(l):
    """ Sort the given list in the way that humans expect.
    """
    l.sort(key=alphanum_key)

In [ ]:
#generate Video
import os
import cv2
def generate_video(): 
    image_folder = '/content/gdrive/MyDrive/Alphapose/AlphaPose/examples/res/Output_Frames' # make sure to use your folder 
    video_name = 'full_scale_video_integration.avi'
    os.chdir("/content/gdrive/MyDrive/Alphapose/AlphaPose/examples/res/") 
      
    images = [img for img in os.listdir(image_folder) 
              if img.endswith(".jpg") or
                 img.endswith(".jpeg") or
                 img.endswith("png")] 
    sort_nicely(images)
    # Array images should only consider 
    # the image files ignoring others if any 
    print(images)  
  
    frame = cv2.imread(os.path.join(image_folder, images[0])) 
  
    # setting the frame width, height width 
    # the width, height of first image 
    height, width, layers = frame.shape   
  
    video = cv2.VideoWriter(video_name, 0, 1, (width, height))  
  
    # Appending the images to the video one by one 
    for image in images:  
        video.write(cv2.imread(os.path.join(image_folder, image)))  
      
    # Deallocating memories taken for window creation 
    cv2.destroyAllWindows()  
    video.release()  # releasing the video generated 
  
  
# Calling the generate_video function 
generate_video() 

['frame1.jpg', 'frame2.jpg', 'frame3.jpg', 'frame4.jpg', 'frame5.jpg', 'frame6.jpg', 'frame7.jpg', 'frame8.jpg', 'frame9.jpg', 'frame10.jpg', 'frame11.jpg', 'frame12.jpg', 'frame13.jpg', 'frame14.jpg', 'frame15.jpg', 'frame16.jpg', 'frame17.jpg', 'frame18.jpg', 'frame19.jpg', 'frame20.jpg', 'frame21.jpg', 'frame22.jpg', 'frame23.jpg', 'frame24.jpg', 'frame25.jpg', 'frame26.jpg', 'frame27.jpg', 'frame28.jpg', 'frame29.jpg', 'frame30.jpg', 'frame31.jpg', 'frame32.jpg', 'frame33.jpg', 'frame34.jpg', 'frame35.jpg', 'frame36.jpg', 'frame37.jpg', 'frame38.jpg', 'frame39.jpg', 'frame40.jpg', 'frame41.jpg', 'frame42.jpg', 'frame43.jpg', 'frame44.jpg', 'frame45.jpg', 'frame46.jpg', 'frame47.jpg', 'frame48.jpg', 'frame49.jpg', 'frame50.jpg', 'frame51.jpg', 'frame52.jpg', 'frame53.jpg', 'frame54.jpg', 'frame55.jpg', 'frame56.jpg', 'frame57.jpg', 'frame58.jpg', 'frame59.jpg', 'frame60.jpg', 'frame61.jpg', 'frame62.jpg', 'frame63.jpg', 'frame64.jpg', 'frame65.jpg', 'frame66.jpg', 'frame67.jpg', 'fr

# TROUBLESHOOTING SECTION

DIFFERENT DEBUG PARAMETERS USED DURING DEVELOPMENT

In [ ]:
import json

with open('/content/gdrive/MyDrive/Alphapose/AlphaPose/examples/res/random_videos/1048231489/alphapose-results.json') as f:
  data = json.load(f)



#read each line
person=data[112]
print(person)
#read number of keypoints / divide by 3 - gives number of keypoints

print(len(person['keypoints'])/3)

#read leye-1, reye-2, lshoulder-5, rshoulder-6 coordinates

print('nose-->'+str(person['keypoints'][0])+","+
      
      str(person['keypoints'][(0)+1]))

print('leye-->'+str(person['keypoints'][1*3])+","+
      
      str(person['keypoints'][(1*3)+1]),str(person['keypoints'][(1*3)+2]))

print('reye-->'+str(person['keypoints'][2*3])+","+
      
      str(person['keypoints'][(2*3)+1]),str(person['keypoints'][(2*3)+2]))

print('lshoulder-->'+str(person['keypoints'][5*3])+","+
      
      str(person['keypoints'][(5*3)+1]),str(person['keypoints'][(5*3)+2]))

print('rshoulder-->'+str(person['keypoints'][6*3])+","+
      
      str(person['keypoints'][(6*3)+1]),str(person['keypoints'][(6*3)+2]))

In [ ]:
import cv2
import math
from google.colab.patches import cv2_imshow
vidcap = cv2.VideoCapture('/content/gdrive/MyDrive/Alphapose/AlphaPose/examples/Video/1048231489-preview.mp4')
success,image = vidcap.read()
count = 0
iter=0
json_count=0

window_name = 'Image'
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
thickness = 2
color = (255, 0, 0) 
# while success:
while success :
  cv2.imwrite("frame%d.jpg" % count, image)     
 

  prev_person=data[json_count]
  person=data[json_count]

  print(person['image_id'],prev_person['image_id'],"frame%d.jpg"%count)
  while person['image_id']==prev_person['image_id'] and json_count<113:
        if json_count==112:
            leye_x=person['keypoints'][1*3]
            leye_y=person['keypoints'][(1*3)+1]
            rshoulder_x=person['keypoints'][6*3]-20
            rshoulder_y=person['keypoints'][(6*3)+1]
            x1=math.floor(leye_x+20)
            y1=math.floor(leye_y-(rshoulder_y-leye_y+20))
            x2=math.floor(rshoulder_x-20)
            y2=math.floor(rshoulder_y)
        
            # start_point = (x1,y1) 
            # end_point = (x2,y2)
            if y1<y2:
                start_point = (x1,y1) 
        
                end_point = (x2,y2)
            elif y1>y2:
                start_point = (x1,y2) 
        
                end_point = (x2,y1)
            image = cv2.rectangle(image, start_point, end_point, color, thickness)
            cv2_imshow( image)
            try:
                crop_img = image[ y1:y2,x2:x1]
                cv2_imshow( crop_img)
                # face=cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)
               
                print(mask_image_faster(crop_img))

            except Exception as err:
                print(err)
                json_count+=1
       
            
        print('json count'+ str(json_count))
        json_count+=1



        prev_person=person
        try:
          person=data[json_count]
        except:
          break
  
 
  iter+=1
  count += 1
  success,image = vidcap.read()
print(count)

0


In [ ]:
TROUBLSHOOTING- SATRUDAY

NameError: ignored

In [ ]:
###Automated code
import cv2
import json
import math

with open('/content/gdrive/MyDrive/Alphapose/AlphaPose/examples/res/random_videos/1048231489/alphapose-results.json') as f:
  data = json.load(f)

print(len(data))
from PIL import Image, ImageDraw
total_persons_from_all_frames=len(data)
from google.colab.patches import cv2_imshow
vidcap = cv2.VideoCapture('/content/gdrive/MyDrive/Alphapose/AlphaPose/examples/Video/1048231489-preview.mp4')
success,image = vidcap.read()
count = 0
iter=0
json_count=0
exception_count=0
window_name = 'Image'
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
thickness = 2
color = (255, 0, 0) 
# while success:
while success :
  cv2.imwrite("frame%d.jpg" % count, image)     
 

  prev_person=data[json_count]
  person=data[json_count]
  image_final=image.copy()
  print(person['image_id'],prev_person['image_id'],"frame%d.jpg"%count)
  while person['image_id']==prev_person['image_id']:
        leye_x=person['keypoints'][1*3]
        leye_y=person['keypoints'][(1*3)+1]
        # rshoulder_x=person['keypoints'][6*3]-20
        rshoulder_x=person['keypoints'][6*3]
        rshoulder_y=person['keypoints'][(6*3)+1]
        lshoulder_x=person['keypoints'][5*3]
        lshoulder_y=person['keypoints'][(5*3)+1]
        # x1=math.floor(leye_x+20)
        # x1=math.floor(lshoulder_x+20)
        x1=math.floor(lshoulder_x)
        # y1=math.floor(leye_y-(rshoulder_y-leye_y+20))
        y1=math.floor(leye_y-(rshoulder_y-leye_y))
        # x2=math.floor(rshoulder_x-20)
        x2=math.floor(rshoulder_x)
        y2=math.floor(rshoulder_y)
     
        # start_point = (x1,y1) 
        # end_point = (x2,y2)
        if y1<y2:
            start_point = (x1,y1) 
      
            end_point = (x2,y2)
        elif y1>y2:
            start_point = (x1,y2) 
     
            end_point = (x2,y1)

        try:
            
          if y1<y2:
            print(str(y1),":",str(y2), str(x2),":",str(x1))
            crop_img = image[y1:y2, x2:x1]
        
        #   elif y1<y2 and x1>x2:
        #       print(str(x2),":",str(x1), str(y1),":",str(y2))
        #       crop_img = image[ x2:x1,y1:y2]
          
        #   elif y1>y2 and x1<x2:
        #      print(str(x1),":",str(x2), str(y2),":",str(y1))
        #      crop_img=image[x1:x2,y2:y1 ]
             
          
          else:
              print(str(y2),":",str(y1), str(x2),":",str(x1))
              crop_img=image[y2:y1,x2:x1]
          
        #   print("display crop")
        #   cv2_imshow(crop_img)
          label=(mask_image_faster(crop_img))
          if 'No Mask' in label:
              color=(0, 0,255) 
          else:
              color=(0, 255,0) 
          image_final = cv2.rectangle(image_final, start_point, end_point, color, thickness)
          org = (x1, y1) 
    
          image_final = cv2.putText(image_final, label, org, font,  
                    fontScale, color, thickness, cv2.LINE_AA)
        except Exception as err:
          print(err, str(start_point), str(end_point),"x1-->y1",str(x1),str(y1),"x2-->y2",str(x2),str(y2),"rshoulder_x-->",str(rshoulder_x))
          cv2_imshow( image_final)
          exception_count+=1
        # if iter>6:
        # try:
        #     # print(start_point,end_point,color,thickness)
        #     # cv2_imshow( crop_img)
        #     cv2.imwrite('/content/sample_data/cropped_images/crop_img_'+str(json_count)+'.jpg', crop_img) 
        # except Exception as ex:
        #     print(ex)
        #     exception_count+=1
        #     # continue
        print('json count'+ str(json_count))
        json_count+=1

        
        prev_person=person
        try:
          person=data[json_count]
        except:
          break
  
 
  iter+=1
  count += 1
  print('exception count is' + str(exception_count))
#   cv2_imshow(image)
  cv2.imwrite("/content/sample_data/Output_Frames/frame%d.jpg" % count, image_final)
  success,image = vidcap.read()
  
print(count)


In [ ]:
###Automated code
import cv2
import json
import math

with open('/content/gdrive/MyDrive/Alphapose/AlphaPose/examples/res/random_videos/trafficsignal/alphapose-results.json') as f:
  data = json.load(f)

print(len(data))
from PIL import Image, ImageDraw
total_persons_from_all_frames=len(data)
from google.colab.patches import cv2_imshow
vidcap = cv2.VideoCapture('/content/gdrive/MyDrive/Alphapose/AlphaPose/examples/Video/traffic_trimmed.mp4')
success,image = vidcap.read()
count = 0
iter=0
json_count=0
exception_count=0
window_name = 'Image'
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
thickness = 2
color = (255, 0, 0) 
# while success:
while success :
  cv2.imwrite("frame%d.jpg" % count, image)     
 

  prev_person=data[json_count]
  person=data[json_count]
  image_final=image.copy()
  
  print(person['image_id'],prev_person['image_id'],"frame%d.jpg"%count)
  while person['image_id']==prev_person['image_id']:
        leye_x=person['keypoints'][1*3]
        leye_y=person['keypoints'][(1*3)+1]
        # rshoulder_x=person['keypoints'][6*3]-20
        rshoulder_x=person['keypoints'][6*3]
        rshoulder_y=person['keypoints'][(6*3)+1]
        lshoulder_x=person['keypoints'][5*3]
        lshoulder_y=person['keypoints'][(5*3)+1]
        # x1=math.floor(leye_x+20)
        # x1=math.floor(lshoulder_x+20)
        x1=math.floor(lshoulder_x)
        # y1=math.floor(leye_y-(rshoulder_y-leye_y+20))
        y1=math.floor(leye_y-(rshoulder_y-leye_y))
        # x2=math.floor(rshoulder_x-20)
        x2=math.floor(rshoulder_x)
        y2=math.floor(rshoulder_y)
        show_image=False
        # start_point = (x1,y1) 
        # end_point = (x2,y2)
        if x1==x2:
             show_image=True
             x1 +=50
             x2 -=50
        if y1<y2:
            start_point = (x1,y1) 
      
            end_point = (x2,y2)
        elif y1>y2:
            start_point = (x1,y2) 
     
            end_point = (x2,y1)

        try:
        
          if y1<y2:
            print(str(y1),":",str(y2), str(x2),":",str(x1))
            crop_img = image[y1:y2, x2:x1]
            # if(show_image):
            #     cv2_imshow(crop_img)

          else:
              print(str(y2),":",str(y1), str(x2),":",str(x1))
              crop_img=image[y2:y1,x2:x1]
          
        #   print("display crop")
        #   cv2_imshow(crop_img)
          
          label=(mask_image_faster(crop_img))
          if 'No Mask' in label:
              color=(0, 0,255) 
          else:
              color=(0, 255,0) 
        #   image_final = cv2.rectangle(image_final, start_point, end_point, color, thickness)
        #   if(show_image):
          image_final = cv2.rectangle(image_final, start_point, end_point, color, thickness)
          org = (x1, y1) 
        
          image_final = cv2.putText(image_final, label, org, font,  
                        fontScale, color, thickness, cv2.LINE_AA)
            # cv2_imshow(image_final)
        except Exception as err:
          print(err, str(start_point), str(end_point),"x1-->y1",str(x1),str(y1),"x2-->y2",str(x2),str(y2),"rshoulder_x-->",str(rshoulder_x))
        #   cv2_imshow( image_final)
          exception_count+=1

        print('json count'+ str(json_count))
        json_count+=1

        
        prev_person=person
        try:
          person=data[json_count]
        except:
          break
  
 
  iter+=1
  count += 1
  print('exception count is' + str(exception_count))

  cv2.imwrite("/content/sample_data/Output_Frames/frame%d.jpg" % count, image_final)
  success,image = vidcap.read()
  
print(count)


In [ ]:
import cv2
from PIL import Image, ImageDraw

from google.colab.patches import cv2_imshow
vidcap = cv2.VideoCapture('/content/gdrive/MyDrive/Alphapose/AlphaPose/examples/Video/79.mp4')
success,image = vidcap.read()
count = 0
iter=0
# while success:
while iter<2:
  cv2.imwrite("frame%d.jpg" % count, image)     # save frame as JPEG file      
  success,image = vidcap.read()
  window_name = 'Image'
    
  # font 
  font = cv2.FONT_HERSHEY_SIMPLEX 
  x41=1115+20
  y41=534-(550-534+20)
  x42=1087-20
  y42=550
  start_point_4 = (x41,y41) 
  end_point_4 = (x42,y42)

# print('leye-->'+str(person['keypoints'][1*3])+","+str(person['keypoints'][(1*3)+1]))

# print('reye-->'+str(person['keypoints'][2*3])+","+str(person['keypoints'][(2*3)+1]))

# print('lshoulder-->'+str(person['keypoints'][5*3])+","+str(person['keypoints'][(5*3)+1]))

# print('rshoulder-->'+str(person['keypoints'][6*3])+","+str(person['keypoints'][(6*3)+1]))

# nose-->447.424072265625,527.6403198242188
# leye-->451.19146728515625,523.8729248046875
# reye-->441.7729797363281,523.8729248046875
# lshoulder-->464.3773498535156,554.0120849609375
# rshoulder-->419.16864013671875,554.0120849609375

# leye_x=person['keypoints'][1*3]
#         leye_y=person['keypoints'][(1*3)+1]
#         rshoulder_x=person['keypoints'][6*3]-20
#         rshoulder_y=person['keypoints'][(6*3)+1]
#         x1=math.floor(leye_x+20)
#         y1=math.floor(leye_y-(rshoulder_y-leye_y+20))
#         x2=math.floor(rshoulder_x-20)
#         y2=math.floor(rshoulder_y)

# nose-->580.2733764648438,502.3037109375
# leye-->587.6251831054688,494.9519348144531
# reye-->572.921630859375,494.9519348144531
# lshoulder-->583.9492797851562,542.7384033203125
# rshoulder-->503.07989501953125,542.7384033203125


# leye-->1630.1387939453125,688.5223999023438
# reye-->1636.74560546875,689.8438110351562
# lshoulder-->1619.56787109375,590.7412719726562
# rshoulder-->1649.959228515625,588.0985107421875

# x1=1650 -----------x1=607
# x2=1629  -----------x2=483
# y2=588----------------y2=542
# y1=688-(588-688+20)=768-----y1= 426
start_point = (x1,y1) (1650,1710) 
  end_point = (x2,y2) (1629,588)
  image[ y1:y2,x2:x1]
   image[ 768:588,1629:1650] [ 426:542, 483,607]

  x31=451+20
  y31=523-(554-523+20)
  x32=419-20
  y32=554
  start_point_3 = (x31,y31) 
  end_point_3 = (x32,y32)

  x01=587+20
  y01=494-(542-494+20)
  x02=503-20
  y02=542
  start_point_0 = (x01,y01) 
  end_point_0 = (x02,y02)

  x1=876+20
  y1=546-(581-546+20)
  x2=791-20
  y2=575
  start_point = (x1,y1) 
  end_point = (x2,y2)

  x11=1014+20
  y11=517-(547-517+20)
  x12=984-20
  y12=551
  start_point_2 = (x11,y11) 
  end_point_2 = (x12,y12)
  # fontScale 
  fontScale = 1
    
  # Blue color in BGR 

    
  # Line thickness of 2 px 
  thickness = 2
    
  # Using cv2.putText() method 
  image = cv2.rectangle(image, start_point_0, end_point_0, color, thickness)
  image = cv2.rectangle(image, start_point, end_point, color, thickness)
  image = cv2.rectangle(image, start_point_2, end_point_2, color, thickness)
  image = cv2.rectangle(image, start_point_3, end_point_3, color, thickness)
  image = cv2.rectangle(image, start_point_4, end_point_4, color, thickness)
  # print('Read a new frame: ', success)
  cv2_imshow( image)
  cv2.imwrite('full.jpg', image) 
  # crop_img = image[ 546-40:575+20,791-20:876+20]
  crop_img = image[ y1:y2,x2:x1]
  crop_img_2 = image[ y11:y12,x12:x11]
  crop_img_0 = image[ y01:y02,x02:x01]
  crop_img_3 = image[ y31:y32,x32:x31]
  crop_img_4 = image[ y41:y42,x42:x41]
  cv2_imshow( crop_img)
  cv2.imwrite('crop_img_1.jpg', crop_img) 
  cv2_imshow( crop_img_2)
  cv2.imwrite('crop_img_2.jpg', crop_img_2) 
  cv2_imshow( crop_img_0)
  cv2.imwrite('crop_img_0.jpg', crop_img_0) 
  cv2_imshow( crop_img_3)
  cv2.imwrite('crop_img_3.jpg', crop_img_3) 
  cv2_imshow( crop_img_4)
  cv2.imwrite('crop_img_4.jpg', crop_img_4) 
  
  iter+=1
  count += 1
print(count)

In [ ]:
###Automated code
import cv2
import json
import math

with open('/content/gdrive/MyDrive/Alphapose/AlphaPose/examples/res/random_videos/trafficsignal/alphapose-results.json') as f:
  data = json.load(f)

print(len(data))
from PIL import Image, ImageDraw
total_persons_from_all_frames=len(data)
from google.colab.patches import cv2_imshow
vidcap = cv2.VideoCapture('/content/gdrive/MyDrive/Alphapose/AlphaPose/examples/Video/traffic_trimmed.mp4')
success,image = vidcap.read()
count = 0
iter=0
json_count=0
exception_count=0
window_name = 'Image'
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
thickness = 2
color = (255, 0, 0) 
# while success:
while success :
  cv2.imwrite("/content/sample_data/frame%d.jpg" % count, image)     
  success,image = vidcap.read()
  count+=1
  
print(count)


# WEB CAM EXPERIMENT - NOT INTEGRATED INTO CODE FULLY

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
while 1:
    try:
        filename = take_photo()
        print('Saved to {}'.format(filename))
        
        # Show the image which was just taken.
        display(Image(filename))
    except Exception as err:
    # Errors will be thrown if the user does not have a webcam or if they do not
    # grant the page permission to access it.
        print(str(err))